In [80]:
import ggpyscraper

ggpyscraper.player.Player(game = 'starcraft2', name = 'Elazer', user = '1221').get_info()

{'id': 'Elazer',
 'image': 'ELAZER-01.jpg',
 'name': 'Mikołaj Ogonowski',
 'givenname': 'Mikołaj',
 'familyname': 'Ogonowski',
 'birth_date': '1997-12-11',
 'country': 'Poland',
 'race': 'Zerg',
 'ids': 'LingKing',
 'aligulac': '5847',
 'eslgaming': '7250354',
 'challonge': 'elazer',
 'tlstream': '',
 'twitch': 'elazer',
 'youtube': '',
 'twitter': 'Elazersc2',
 'facebook': 'ElazerSC',
 'instagram': 'elazersc2',
 'fanclub': '',
 'team_history': Empty DataFrame
 Columns: []
 Index: []}

In [42]:
import pandas as pd

url = "https://liquipedia.net/starcraft2/Serral/Matches"

tables = pd.read_html(url)
tables[1]

,Date,Tier,Unnamed: 2,Tournament,Participant,Score,vs. Opponent,VOD(s)
0,"January 4, 2026 - 18:00 CET",2v2 (B),NaN,uThermal 2v2 Jan 2026,SerralZhuGeLiang,1 : 2,SKillousElazer,NaN
1,"January 4, 2026",2v2 (B),NaN,uThermal 2v2 Jan 2026,SerralZhuGeLiang,2 : 0,SpazymazyUrono,NaN
2,"January 4, 2026",2v2 (B),NaN,uThermal 2v2 Jan 2026,SerralZhuGeLiang,2 : 1,triggerClem,NaN
3,"December 12, 2025 - 19:45 CET",Showm. (B),NaN,Basilisk Big Brain Bouts #101 - The Basilisk One,Serral,3 : 0,ShoWTimE,NaN
4,"October 31, 2025 - 22:00 KST",A-Tier,NaN,CranK Gathers Season 2: SC II Pro Teams - Play...,BSLSK,4 : 7,TL,NaN
...,...,...,...,...,...,...,...,...
245,"May 2, 2023 - 16:00 CEST",S-Tier,NaN,ESL Summer: Europe - Gr A R 1,Serral,2 : 0,Spatz,NaN
246,"April 28, 2023 - 22:05 CST",S-Tier,NaN,WTL 2023 Summer - Round 2,BSLSK,4 : 2,PSIS,NaN
247,"April 18, 2023 - 20:00 CST",S-Tier,NaN,WTL 2023 Summer - Round 1,BSLSK,4 : 2,ABS,NaN
248,"April 17, 2023 - 17:05 CEST",Week. (B),NaN,KFC2023#1,Serral,3 : 0,Oliveira,NaN


In [39]:
tables[0].iloc[0].tolist()

['Currently, Player Matches Tables are limited to 250 matches due to runtime limitations.']

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin
import time

BASE_URL = "https://liquipedia.net"
PLAYER_NAME = "Elazer"
GAME = "starcraft2"
HEADERS = {"User-Agent": "Mozilla/5.0"}

replace_wins_dict = {
    'W' : 1,
    'L' : 0,
    '' : 0
}

def get_tables_from_url(url):
    resp = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(resp.text, "html.parser")
    tables = soup.find_all("table", class_="wikitable")
    
    data = []
    for table in tables:
        rows = table.find_all("tr")[1:]
        for r in rows:
            cols = [c.get_text(strip=True) for c in r.find_all("td")]
            if cols:
                data.append(cols)
            # end if
        # end for
    # end for
    return data
# end def

def parse_liquipedia_datetime(s: pd.Series) -> pd.Series:
    s = s.str.replace(
        r'(UTC|CET|CEST|KST|EDT|CDT|CST|EST|PDT)$',
        '',
        regex=True
    ).str.strip()

    s = s.str.replace(r'\s*-\s*\d{1,2}:\d{2}$', '', regex=True)

    return pd.to_datetime(s, format='mixed', errors='coerce').dt.normalize()
# end def

def extract_tier(s: pd.Series) -> pd.Series:
    tier = s.str.extract(r'\(([SABC])\)', expand=False)

    tier = tier.fillna(
        s.str.extract(r'\b([SABC])\s*-?\s*Tier\b', expand=False)
    )

    return tier
# end def

def get_matcher_per_player(player_name):
    player_url = f"{BASE_URL}/{GAME}/{player_name}/Matches"
    all_matches = get_tables_from_url(player_url)

    resp = requests.get(player_url, headers=HEADERS)
    soup = BeautifulSoup(resp.text, "html.parser")
    links = soup.find_all("a")

    season_links = set()
    for a in links:
        href = a.get("href")
        if href and "Matches" in href and href != f"/{GAME}/{PLAYER_NAME}/Matches":
            full_url = urljoin(BASE_URL, href)
            season_links.add(full_url)
        # end if
    # end for

    for link in season_links:
        try:
            all_matches.extend(get_tables_from_url(link))
            time.sleep(1)
        except Exception as e:
            print("Error:", e)
        # end try
    # end for

    df = pd.DataFrame(all_matches)
    df.columns = ('date', 'match', 'skip1', 'tournament', 'player', 'score', 'oponent', 'skip2')
    df = df.drop(columns = ['skip1', 'skip2'])
    df = df[df['player'] == PLAYER_NAME].reset_index(drop = True)

    df['player_wins'] = df['score'].apply(lambda x : x.split(':')[0].strip())
    df['oponent_wins'] = df['score'].apply(lambda x : x.split(':')[-1].strip())
    df[['player_wins', 'oponent_wins']] = df[['player_wins', 'oponent_wins']].replace(replace_wins_dict)
    df[['player_wins', 'oponent_wins']] = df[['player_wins', 'oponent_wins']].astype(int)
    df['best_of'] = (
        df[['player_wins', 'oponent_wins']].max(axis = 1)
            .pipe(lambda s: s + 1 + (s % 2))
    )

    df['date'] = parse_liquipedia_datetime(df['date'])
    df['day'] = df['date'].apply(lambda x : x.day)
    df['month'] = df['date'].apply(lambda x : x.month)
    df['year'] = df['date'].apply(lambda x : x.year)

    df['tournament_tier'] = extract_tier(df['match'])

    df = df.drop(columns = ('score'))

    # need Match_history here

    series_list = []
    for row in df.iterrows():
        row_tmp = row[1].copy()
        row_tmp['player'], row_tmp['oponent'] = row_tmp['oponent'], row_tmp['player']
        row_tmp['player_wins'], row_tmp['oponent_wins'] = row_tmp['oponent_wins'], row_tmp['player_wins']
        series_list.append(row_tmp)
    # end for

    df_tmp = pd.DataFrame(series_list)
    df = pd.concat([df, df_tmp]).reset_index(drop = True)
    
    return df
# end def

df = get_matcher_per_player('Elazer')
df

,date,match,tournament,player,oponent,player_wins,oponent_wins,best_of,day,month,year,tournament_tier
0,2025-12-26,Showm. (B),Basilisk Big Brain Bouts #103 - The Good One,Elazer,Nicoract,1,3,5,26,12,2025,B
1,2025-12-22,Week. (B),WardiTV Mondays #66,Elazer,Classic,2,3,5,22,12,2025,B
2,2025-12-22,Week. (B),WardiTV Mondays #66,Elazer,MaxPax,2,0,3,22,12,2025,B
3,2025-12-22,Week. (B),WardiTV Mondays #66,Elazer,SHIN,2,0,3,22,12,2025,B
4,2025-12-22,Week. (B),WardiTV Mondays #66,Elazer,trigger,2,1,3,22,12,2025,B
...,...,...,...,...,...,...,...,...,...,...,...,...
1033,2018-07-24,Qual. (S),2018 WCS Montreal - Europe: Open Qualifiers - ...,Lillekanin,Elazer,1,2,3,24,7,2018,S
1034,2018-07-24,Qual. (S),2018 WCS Montreal - Europe: Open Qualifiers - ...,Brat_OK,Elazer,0,2,3,24,7,2018,S
1035,2018-07-24,Qual. (S),2018 WCS Montreal - Europe: Open Qualifiers - ...,Denver,Elazer,2,1,3,24,7,2018,S
1036,2018-07-24,Qual. (S),2018 WCS Montreal - Europe: Open Qualifiers - ...,Guru,Elazer,0,2,3,24,7,2018,S


In [ ]:
q = '''
https://liquipedia.net/starcraft2/Special:RunQuery/Match_history?title=Special%3ARunQuery%2FMatch_history&Head_to_head_query=player%3DElazer%26opponent%3DClassic_%2528Kim_Doh_Woo%2529%26opponentcountry%3D%26game%3DLegacy%2Bof%2Bthe%2BVoid%26ltier%3D%26sdate%255Bday%255D%3D21%26sdate%255Bmonth%255D%3D12%26sdate%255Byear%255D%3D2021%26edate%255Bday%255D%3D21%26edate%255Bmonth%255D%3D12%26edate%255Byear%255D%3D2025%26walkover%3D%26matchups%3D%26maps%3D&pfRunQueryFormName=Match+history&wpRunQuery=&pf_free_text=&Head+to+head+query%5Bplayer%5D=Elazer&Head+to+head+query%5Bopponent%5D=Classic_%28Kim_Doh_Woo%29&Head+to+head+query%5Bopponentcountry%5D=&Head+to+head+query%5Bgame%5D=Legacy+of+the+Void&Head+to+head+query%5Bltier%5D=&Head+to+head+query%5Bsdate%5D%5Bday%5D=21&Head+to+head+query%5Bsdate%5D%5Bmonth%5D=12&Head+to+head+query%5Bsdate%5D%5Byear%5D=2023&Head+to+head+query%5Bedate%5D%5Bday%5D=21&Head+to+head+query%5Bedate%5D%5Bmonth%5D=12&Head+to+head+query%5Bedate%5D%5Byear%5D=2025&Head+to+head+query%5Bwalkover%5D=&Head+to+head+query%5Bmatchups%5D=&Head+to+head+query%5Bmaps%5D=&wpRunQuery=&pf_free_text=
'''

,date,match,tournament,player,oponent,player_wins,oponent_wins,best_of,day,month,year,tournament_tier
0,2025-12-26,Showm. (B),Basilisk Big Brain Bouts #103 - The Good One,Elazer,Nicoract,1,3,5,26,12,2025,B
519,2025-12-26,Showm. (B),Basilisk Big Brain Bouts #103 - The Good One,Nicoract,Elazer,3,1,5,26,12,2025,B


In [ ]:
def get_player_race(player_url):
    resp = requests.get(player_url, headers=HEADERS)
    soup = BeautifulSoup(resp.text, "html.parser")

    for div in soup.select("div.infobox-cell-2.infobox-description"):
        if div.get_text(strip=True).lower().startswith("race"):
            value_div = div.find_next_sibling("div")
            if value_div:
                return value_div.get_text(strip=True)
            # end if
        # end if
    # end for

    return None
# end def


PLAYER_MAIN_URL = f"{BASE_URL}/{GAME}/{PLAYER_NAME}".replace('Elazer', 'ByuN')
player_race = get_player_race(PLAYER_MAIN_URL)
print(player_race)

Terran


In [134]:
df['match'].value_counts()

match
B-Tier        107
S-Tier         92
A-Tier         91
Qual. (S)      76
Qual. (B)      44
Week. (B)      37
Mon. (B)       21
Qual. (A)      21
Showm. (B)      9
C-Tier          7
Biw. (C)        7
Biw. (B)        5
Showm. (C)      2
Name: count, dtype: int64

In [125]:
df['oponent_wins'].value_counts().index

Index(['0', '2', '1', '3', '4', 'L', '5', ''], dtype='object', name='oponent_wins')

In [99]:
def get_player_race(player_url):
    resp = requests.get(player_url, headers=HEADERS)
    soup = BeautifulSoup(resp.text, "html.parser")

    for div in soup.select("div.infobox-cell-2.infobox-description"):
        if div.get_text(strip=True).lower().startswith("race"):
            value_div = div.find_next_sibling("div")
            if value_div:
                return value_div.get_text(strip=True)
            # end if
        # end if
    # end for

    return None
# end def


PLAYER_MAIN_URL = f"{BASE_URL}/{GAME}/{PLAYER_NAME}".replace('Elazer', 'ByuN')
player_race = get_player_race(PLAYER_MAIN_URL)
print(player_race)

Terran
